<a href="https://colab.research.google.com/github/marcusflygar1-hash/AH2179_Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Okey-



Psuedo Kod.
rensa och "clean" datan. Ta bort all null values och NaN's.

Dela sedan upp i 5 minuters tidsperioder.



## Setting up the data and cleaning etc.

In [ ]:
tds = pd.read_csv('training_dataset.csv', sep=';')
ev_ds = pd.read_csv('evaluation_dataset.csv',sep=';')
f_ev_ds = pd.read_csv('final_evaluation_dataset.csv', sep=';')


In [ ]:
# Make sure Date is a string before concatenating
tds["Date"] = tds["Date"].astype(str)

# Combine Date and Time into a single datetime column
tds["datetime"] = pd.to_datetime(tds["Date"] + " " + tds["Time"])

# Plot speed vs datetime
tds.plot(x="datetime", y="SPEED_MS_AVG", kind="line", figsize=(12,6))
plt.xlabel("Time")
plt.ylabel("Average Speed (m/s)")
plt.title("Speed over Time")
plt.show()

In [ ]:
tds.plot(x='datetime', y='FLOW', kind='line', figsize=(20,14))
plt.xlabel('Time')
plt.ylabel('Flow')
plt.title('Flow over Time')

In [ ]:
print('\n training dataset')
print(tds.head())
print('\n evaluation dataset------------')
print(ev_ds.head())
print('\n final evaluation dataset --------')
print(f_ev_ds.head())

In [ ]:
print('Training dataset')
print(tds.isna().sum())
print('\n eval dataset')
print(ev_ds.isna().sum())
print('\n final eval dataset')
print(f_ev_ds.isna().sum())

In [ ]:
#drop all NaN values.
f_ev_ds.dropna(inplace=True)
tds.dropna(inplace=True)
ev_ds.dropna(inplace=True)

In [ ]:
print('Training dataset')
print(tds.isna().sum())
print('\n eval dataset')
print(ev_ds.isna().sum())
print('\n final eval dataset')
print(f_ev_ds.isna().sum())

In [ ]:
tot_days = tds['datetime'].dt.date.nunique()
print(f'the total amount of days in the dataset is: {tot_days}')
# tot_speed = tds['SPEED_MS_AVG'].sum()
# print(f'\n The total avg_speed in m/s is : {tot_speed}')
tot_flow = tds['FLOW'].sum()
print(f'\n the total flow of the dataset is:{tot_flow}')


We only find nan values in the flow category.

In [ ]:
#define what is congestion and what is not.
# congestion_flow = tds[tds['FLOW'] < 100]
# congestion_speed = tds[tds['SPEED_MS_AVG'] < 10]

In [ ]:
# filtering the dataset to only include the data within hours 04:00 and 06:00, this to get an accurate description of free flow speed.
tds_offpeak = tds[(tds['datetime'].dt.time >= pd.to_datetime("04:00:00").time()) &
                (tds['datetime'].dt.time <= pd.to_datetime("06:00:00").time())]

#Ensuring we only get correctly read speeds. E.g Removing any negative speeds and random slow drivers
#that do not actually depict the actual free flowspeed e.g people speeding and drinving super slow...
tds_offpeak = tds_offpeak[(tds_offpeak['FLOW'] > 0) &
                        (tds_offpeak['SPEED_MS_AVG'].between(5, 50))]

# Calculating the free flow speed t
# only take the observations in the 85th quantile [m/s]
ffs_ms = (tds_offpeak['SPEED_MS_AVG'].quantile(0.85))
# convert to km/h
ffs_kmh = (ffs_ms * 3.6)
print(f"Free-flow speed (85th percentile): \n Free flow speed: {ffs_ms} [m/s] \n Free flow speed {ffs_kmh} [km/h]")